In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from numpy import pi

from helpers.datasets.make_and_save.aggregated_signal import Aggregated_Signal_Dataframe_Handler
from helpers.experiment.constants import Paths_to_Directories
from helpers.datasets.constants import Names_of_Levels, Raw_Signal_Trial_Ranges, Names_of_Variables, Names_of_Labels
from helpers.plot.util import to_x10_notation



def reduce_to_num_per_label(dataframe, num):
    get_subset = lambda x : x.iloc[:num]
    dataframe = (
        dataframe.groupby(Names_of_Labels().unbinned, group_keys=False)[dataframe.columns]
        .apply(get_subset)
    )
    return dataframe


def get_min_over_labels(dataframe):
    return dataframe[Names_of_Labels().unbinned].value_counts().values.min()





def plot_distribution(data, ax, delta_C9_value, cmap, norm, xlimits, num_bins):

    is_standard_model = (delta_C9_value == 0)

    color = cmap(norm(delta_C9_value)) if not is_standard_model else "dimgrey"
    linestyle = 'solid' if not is_standard_model else (0, (1, 1))
    label = None if not is_standard_model else r"SM ($\delta C_9 = 0$)"
    zorder = None if not is_standard_model else 10
    
    ax.hist(
        data, 
        range=xlimits,
        histtype="step",
        density=True, 
        bins=num_bins, 
        color=color,
        linestyle=linestyle,
        label=label,
        zorder=zorder
    )


def plot_variable(dataframe, name_of_variable, level, ax, cmap, norm, num_bins):

    assert name_of_variable in Names_of_Variables().tuple_
    
    xlabels = {
        Names_of_Variables().q_squared : r"$q^2$ [GeV$^2$]",
        Names_of_Variables().cos_theta_mu : r"$\cos\theta_\mu$",
        Names_of_Variables().cos_k : r"$\cos\theta_K$",
        Names_of_Variables().chi : r"$\chi$"
    }
    xlimits = {
        Names_of_Variables().q_squared : (0, 20),
        Names_of_Variables().cos_theta_mu : (-1, 1),
        Names_of_Variables().cos_k : (-1, 1),
        Names_of_Variables().chi : (0, 2*pi),
    }
    ylims = {
        Names_of_Variables().q_squared : (0, 0.14),
        Names_of_Variables().cos_theta_mu : (0, 0.75),
        Names_of_Variables().cos_k : (0, 0.85),
        Names_of_Variables().chi : (0, 0.195),
    }

    for delta_C9_value, dataframe_subset in dataframe.groupby(Names_of_Labels().unbinned):
        plot_distribution(
            data=dataframe_subset[name_of_variable],
            ax=ax,
            delta_C9_value=delta_C9_value,
            cmap=cmap,
            norm=norm,
            xlimits=xlimits[name_of_variable],
            num_bins=num_bins
        )

    if name_of_variable == Names_of_Variables().chi:
        ax.set_xticks(
            [0, pi, 2*pi], 
            ["0", r"$\pi$", r"$2 \pi$"],
        )
    ax.set_xlabel(xlabels[name_of_variable], fontsize=12.5)
    ax.set_ylim(ylims[name_of_variable])
    ax.locator_params(axis='y', tight=True, nbins=2)


def plot_all_variables(fig, axs_4x2, dataframe_generator, dataframe_detector, num_bins):

    print(        
        r"Generator events/$\delta C_9$: " + f"{to_x10_notation(len(dataframe_generator)/44)}"
        "\nDetector events/" + r"$\delta C_9$: " + f"{to_x10_notation(len(dataframe_detector)/44)}"
    )
    
    cmap = plt.cm.coolwarm
    norm = mpl.colors.CenteredNorm(
        vcenter=0, 
        halfrange=abs(dataframe_generator[Names_of_Labels().unbinned].min())
    )

    for variable, ax in zip(Names_of_Variables().tuple_, axs_4x2[:,0].flat):
        plot_variable(
            dataframe=dataframe_generator,
            name_of_variable=variable,
            level="generator",
            ax=ax,
            cmap=cmap,
            norm=norm,
            num_bins=num_bins
        )

    for variable, ax in zip(Names_of_Variables().tuple_, axs_4x2[:,1].flat):
        plot_variable(
            dataframe=dataframe_detector,
            name_of_variable=variable,
            level="detector",
            ax=ax,
            cmap=cmap,
            norm=norm,
            num_bins=num_bins
        )

    axs_4x2.flat[0].legend()

    axs_4x2[0,0].set_title("Generator\n", fontsize=14)
    axs_4x2[0,1].set_title("Detector\n", fontsize=14)

    cbar = fig.colorbar(
        mpl.cm.ScalarMappable(norm=norm, cmap=cmap), 
        ax=axs_4x2, 
        orientation='horizontal', 
        shrink=0.9
    )
    cbar.set_label(r'$\delta C_9$', size=13.5)








mpl.rcParams["figure.dpi"] = 400
mpl.rcParams["axes.titlesize"] = 8
mpl.rcParams["figure.titlesize"] = 8
mpl.rcParams["figure.labelsize"] = 30
mpl.rcParams["text.usetex"] = True
mpl.rcParams["text.latex.preamble"] = r"\usepackage{bm}\usepackage{amsmath}"
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.serif"] = ["Computer Modern"]
mpl.rcParams["font.size"] = 8
mpl.rcParams["axes.titley"] = None
mpl.rcParams["axes.titlepad"] = 2
mpl.rcParams["legend.fancybox"] = False
mpl.rcParams["legend.framealpha"] = 0
mpl.rcParams["legend.markerscale"] = 1
mpl.rcParams["legend.fontsize"] = 11.5


list_of_variable_names = Names_of_Variables().list_ + [var + "_mc" for var in Names_of_Variables().list_]

df_gen = Aggregated_Signal_Dataframe_Handler(
    path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
    level=Names_of_Levels().generator,
    trial_range=range(1,41)
).get_dataframe()

df_det = Aggregated_Signal_Dataframe_Handler(
    path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
    level=Names_of_Levels().detector,
    trial_range=range(1,41)
).get_dataframe()
    
# num_lowest = min(get_min_over_labels(df_det), get_min_over_labels(df_gen))
# df_gen = reduce_to_num_per_label(df_gen, num=num_lowest)
# df_det = reduce_to_num_per_label(df_det, num=num_lowest)


Distributions

In [ ]:
mpl.rcParams["figure.figsize"] = (6, 8)


fig, axs = plt.subplots(
    4, 2, 
    sharey="row", 
    layout="compressed"
)

plot_all_variables(fig, axs, dataframe_generator=df_gen, dataframe_detector=df_det, num_bins=30)



plt.savefig(Paths_to_Directories().path_to_plots_dir.joinpath("variables.png"), bbox_inches="tight")
plt.close()

Generator events/$\delta C_9$: $ 3.4\times 10^{5}$
Detector events/$\delta C_9$: $ 3.4\times 10^{5}$


Asymmetries

In [ ]:
from helpers.plot.asymmetries_plot import plot_afb_and_s5
from helpers.variable_calculations.asymmetries_calc import calc_afb_of_q_squared_over_delta_C9_values, calc_s5_of_q_squared_over_delta_C9_values



def plot_afb(afb_results, ax, cmap, norm, alpha):
    
    for delta_C9_value, bin_middles, afbs, afb_errs in zip(
        afb_results.delta_C9_values,
        afb_results.bin_middles_over_delta_C9_values,
        afb_results.afbs_over_delta_C9_values,
        afb_results.afb_errors_over_delta_C9_values
    ):
        
        is_standard_model = (delta_C9_value == 0)
        color = cmap(norm(delta_C9_value), alpha=alpha) if not is_standard_model else "dimgrey"
        label = None if not is_standard_model else r"SM ($\delta C_9 = 0$)"
        zorder = None if not is_standard_model else 10
        
        ax.scatter(
            bin_middles, 
            afbs, 
            label=label,
            color=color, 
            edgecolors='none',
            s=10,
            zorder=zorder,
        )
        ax.errorbar(
            bin_middles, 
            afbs, 
            yerr=afb_errs, 
            fmt='none', 
            ecolor=color, 
            elinewidth=0.5, 
            capsize=0, 
            zorder=zorder,
        )

    ax.set_ylim(-0.25, 0.46)


def plot_s5(s5_results, ax, cmap, norm, alpha):

    for delta_C9_value, bin_middles, s5s, s5_errs in zip(
        s5_results.delta_C9_values,
        s5_results.bin_middles_over_delta_C9_values,
        s5_results.s5s_over_delta_C9_values,
        s5_results.s5_errors_over_delta_C9_values
    ):
        
        is_standard_model = (delta_C9_value == 0)
        color = cmap(norm(delta_C9_value), alpha=alpha) if not is_standard_model else "dimgrey"
        label = None if not is_standard_model else r"SM ($\delta C_9 = 0$)"
        zorder = None if not is_standard_model else 10

        ax.scatter(
            bin_middles, 
            s5s, 
            label=label,
            color=color, 
            edgecolors='none',
            s=10,
            zorder=zorder,
        )
        ax.errorbar(
            bin_middles, 
            s5s, 
            yerr=s5_errs, 
            fmt='none', 
            ecolor=color, 
            elinewidth=0.5, 
            capsize=0, 
            zorder=zorder,
        )
    
    ax.set_ylim(-0.48, 0.38)



def plot_afb_and_s5(
    fig, 
    axs_2x2, 
    afb_results_generator, 
    afb_results_detector,
    s5_results_generator,
    s5_results_detector, 
    alpha=0.85
):
    
    def get_colorbar_halfrange(afb_results, s5_results):
        assert min(afb_results.delta_C9_values) == min(s5_results.delta_C9_values)
        return abs(min(afb_results.delta_C9_values))

    cmap = plt.cm.coolwarm
    norm = mpl.colors.CenteredNorm(
        vcenter=0, 
        halfrange=get_colorbar_halfrange(afb_results=afb_results_generator, s5_results=s5_results_generator),
    )

    for ax, afb_results in zip(axs_2x2[0,:], (afb_results_generator, afb_results_detector)):
        plot_afb(afb_results=afb_results, ax=ax, cmap=cmap, norm=norm, alpha=alpha)
    
    for ax, s5_results in zip(axs_2x2[1,:], (s5_results_generator, s5_results_detector)):
        plot_s5(s5_results=s5_results, ax=ax, cmap=cmap, norm=norm, alpha=alpha)

    cbar = fig.colorbar(
        mpl.cm.ScalarMappable(norm=norm, cmap=cmap), 
        ax=axs_2x2, 
        orientation='horizontal',
        location="bottom", 
        shrink=0.9,
        pad=0.15,
    )

    cbar.set_label(r'$\delta C_9$', fontsize=13.5)


In [ ]:
s5_results_gen = calc_s5_of_q_squared_over_delta_C9_values(df_gen, 50)
s5_results_det = calc_s5_of_q_squared_over_delta_C9_values(df_det, 50)
afb_results_gen = calc_afb_of_q_squared_over_delta_C9_values(df_gen, 50)
afb_results_det = calc_afb_of_q_squared_over_delta_C9_values(df_det, 50)

In [ ]:
mpl.rcParams["figure.figsize"] = (6, 6)


fig, axs = plt.subplots(
    2, 2, 
    sharex=True,
    sharey="row", 
    layout='compressed',
)

plot_afb_and_s5(fig, axs, afb_results_generator=afb_results_gen, afb_results_detector=afb_results_det, s5_results_generator=s5_results_gen, s5_results_detector=s5_results_det)

axs.flat[0].legend(markerscale=2)
axs[0,0].set_ylabel("$A_{FB}$", fontsize=14)
axs[1,0].set_ylabel("$S_5$", fontsize=14)
axs[0,0].set_title("Generator\n", fontsize=14)
axs[0,1].set_title("Detector\n", fontsize=14)
fig.supxlabel(r"$q^2$" + r" [GeV$^2$]", fontsize=14, x=0.543, y=0.18)


plt.savefig(Paths_to_Directories().path_to_plots_dir.joinpath("asymmetries.png"), bbox_inches="tight")
plt.close()


Resolutions

In [2]:
from helpers.variable_calculations.resolutions_calc import calculate_resolution
from helpers.plot.util import stats_legend


list_of_variable_names = Names_of_Variables().list_ + [var + "_mc" for var in Names_of_Variables().list_]

df_gen = Aggregated_Signal_Dataframe_Handler(
    path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
    level=Names_of_Levels().generator,
    trial_range=range(1,41)
).get_dataframe()

df_det = Aggregated_Signal_Dataframe_Handler(
    path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
    level=Names_of_Levels().detector,
    trial_range=range(1,41)
).get_dataframe()
    
num_lowest = min(get_min_over_labels(df_det), get_min_over_labels(df_gen))
df_gen = reduce_to_num_per_label(df_gen, num=num_lowest)
df_det = reduce_to_num_per_label(df_det, num=num_lowest)

In [3]:
resolutions = {var : calculate_resolution(df_det, var) for var in Names_of_Variables().list_}

mins = [resolutions[var].abs().max() for var in Names_of_Variables().list_]

mins


Applying q^2 veto.
Applied q^2 veto.
Applying q^2 veto.
Applied q^2 veto.
Applying q^2 veto.
Applied q^2 veto.
Applying q^2 veto.
Applied q^2 veto.


[np.float64(18.551858032789596),
 np.float64(1.966028871813107),
 np.float64(1.9977443516747821),
 np.float64(3.1415916070365353)]

In [7]:
for var in Names_of_Variables().list_:
    print(len(resolutions[var]))

14980242
14980242
14980242
14980242


In [9]:
fig, axs = plt.subplots(2,2, layout="compressed", sharey=False )

# xlimits = {var : (-0.15*resolutions[var].std(), 0.15*resolutions[var].std()) for var in Names_of_Variables().list_}

xlimits = {
    Names_of_Variables().q_squared : (-0.15, 0.15),
    Names_of_Variables().cos_theta_mu : (-0.015, 0.015),
    Names_of_Variables().cos_k : (-0.06, 0.06),
    Names_of_Variables().chi : (-0.04, 0.04)
}

xlabels = {
    Names_of_Variables().q_squared : r"$q^2_\text{\;det} - q^2_\text{\;gen}$",
    Names_of_Variables().cos_theta_mu : r"$\cos\theta_{\mu\;\text{det}} - \cos\theta_{\mu\;\text{gen}}$",
    Names_of_Variables().cos_k : r"$\cos\theta_{K\;\text{det}} - \cos\theta_{K\;\text{gen}}$",
    Names_of_Variables().chi : r"$\chi_\text{\;det} - \chi_\text{\;gen}$"
}

num_bins = {
    Names_of_Variables().q_squared : 1000,
    Names_of_Variables().cos_theta_mu : 1000,
    Names_of_Variables().cos_k : 1000,
    Names_of_Variables().chi : 1000
}

for ax, var in zip(axs.flat, Names_of_Variables().list_):

    ax.hist(
        resolutions[var],
        bins=num_bins[var],
        range=xlimits[var],  # does this cut out data?
        color="purple",
        histtype="step",
        density=False,
    )
    
    ax.hist(
        resolutions[var],
        label=stats_legend(resolutions[var], show_count=False),
        bins=num_bins[var],
        range=xlimits[var],  # does this cut out data?
        color="purple",
        histtype="step",
        density=False,
        alpha=0
    )

    ax.locator_params(nbins=3)
    ax.legend(markerscale=0.1, fontsize=8, loc="upper right")

    print(ax.get_yticks())

    ax.set_yticks(ax.get_yticks(), [to_x10_notation(tick) for tick in ax.get_yticks()])

    ax.set_xlabel(xlabels[var], fontsize=13)


plt.savefig(Paths_to_Directories().path_to_plots_dir.joinpath("resolutions.png"), bbox_inches="tight")
plt.close()

[     0. 100000. 200000.]
[     0.  50000. 100000. 150000.]
[     0.  50000. 100000. 150000.]
[     0.  50000. 100000.]


Efficiencies

In [2]:
from helpers.variable_calculations.efficiencies_calc import calculate_efficiency
from helpers.plot.efficiencies_plot import plot_efficiency
from numpy import pi

list_of_variable_names = Names_of_Variables().list_ + [var + "_mc" for var in Names_of_Variables().list_]

df_gen = Aggregated_Signal_Dataframe_Handler(
    path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
    level=Names_of_Levels().generator,
    trial_range=range(1,41)
).get_dataframe()

df_det = Aggregated_Signal_Dataframe_Handler(
    path_to_main_datasets_dir=Paths_to_Directories().path_to_main_datasets_dir,
    level=Names_of_Levels().detector,
    trial_range=range(1,41)
).get_dataframe()

In [3]:
bounds = {
    Names_of_Variables().q_squared : (0, 20),
    Names_of_Variables().cos_theta_mu : (-1, 1),
    Names_of_Variables().cos_k : (-1, 1),
    Names_of_Variables().chi : (0, 2*pi)
}

xlabels = {
    Names_of_Variables().q_squared : r"$q^2$ [GeV$^2$]",
    Names_of_Variables().cos_theta_mu : r"$\cos\theta_\mu$",
    Names_of_Variables().cos_k : r"$\cos\theta_K$",
    Names_of_Variables().chi : r"$\chi$"
}

fig, axs = plt.subplots(2, 2, layout="compressed", sharey=True)

for var, ax in zip(Names_of_Variables().list_, axs.flat):

    series_gen = df_gen[var]
    series_det = df_det[var]

    eff = calculate_efficiency(generator_level_series=series_gen, detector_level_series=series_det, num_bins=50, bounds=bounds[var])

    plot_efficiency(eff, ax)
    ax.locator_params(nbins=3)
    ax.set_xlabel(xlabels[var], fontsize=13)
    ax.set_ylim(0, 0.65)

    if var == Names_of_Variables().chi:
        ax.set_xticks([0, pi, 2*pi], [r"$0$", r"$\pi$", r"$2\pi$"])


fig.supylabel("Efficiency " + r"($n_\text{det}/n_\text{gen}$)", fontsize=14, y=0.54, x=-0.05)


plt.savefig(Paths_to_Directories().path_to_plots_dir.joinpath("efficiencies.png"), bbox_inches="tight")
plt.close()

num events generator:  42115273
num events detector:  15493554
num events generator:  42120039
num events detector:  15495427
num events generator:  42120039
num events detector:  15494299
num events generator:  42120039
num events detector:  15494299


In [4]:
15493554/42115273

0.36788444895038436